In [3]:
!pip install requests
!pip install sklearn
!pip install matplotlib
!pip install numpy
!pip install pandas

     ---------------------------------------- 7.6/7.6 MB 252.9 kB/s eta 0:00:00
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ---------------------------------------- 2.5/2.5 MB 268.8 kB/s eta 0:00:00
     ------------------------------------ 429.4/429.4 kB 311.9 kB/s eta 0:00:00
     ---------------------------------------- 55.4/55.4 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 262.6 kB/s eta 0:00:00


In [7]:
!pip install scikit-learn

     ---------------------------------------- 8.4/8.4 MB 246.1 kB/s eta 0:00:00
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
     -------------------------------------- 42.5/42.5 MB 245.7 kB/s eta 0:00:00
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [8]:

import numpy as np
from numpy import genfromtxt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import matplotlib.pyplot as plt

import requests
import csv
import json

In [10]:
# set prediction step and read data
prediction_step = 1

# my_data = genfromtxt("/home/aban/work/Bitcoin/data/data_30_min.csv", delimiter=',')
my_data = genfromtxt("./data_30_min.csv", delimiter=',')
my_data2= my_data[1:(len(my_data)+1),3:6]

# Get previous price
a_high = my_data2[len(my_data2)-1,0]
a_low = my_data2[len(my_data2)-1,1]
a_close = my_data2[len(my_data2)-1,2]

# Convert data to data frame
data_high = pd.DataFrame(my_data2[:,0], columns=["High_price"])
data_low = pd.DataFrame(my_data2[:,1], columns=["Low_price"])
data_close = pd.DataFrame(my_data2[:,2], columns=["Close_price"])

del(my_data,my_data2)
print("Successful")

Successful


In [11]:
# scale data
# scale high
m_high = np.mean(np.array(data_high["High_price"]))
s_high = np.std(np.array(data_high["High_price"]))
a = (data_high["High_price"] - m_high) / s_high
data_high["High_price"] = a
del(a)

# scale low
m_low = np.mean(np.array(data_low["Low_price"]))
s_low = np.std(np.array(data_low["Low_price"]))
a = (data_low["Low_price"] - m_low) / s_low
data_low["Low_price"] = a
del(a)

# scale close
m_close = np.mean(np.array(data_close["Close_price"]))
s_close = np.std(np.array(data_close["Close_price"]))
a = (data_close["Close_price"] - m_close) / s_close
data_close["Close_price"] = a
del(a)
print("Successful")

Successful


In [15]:
# create train and test data
# high data
data_high["Predictin"] = data_high[["High_price"]].shift(-prediction_step)
x_high = np.array(data_high.drop(["Predictin"], 1))
x_high = x_high[:len(data_high)-prediction_step] # remove NA cell
y_high = np.array(data_high["Predictin"])
y_high = y_high[:-prediction_step] # remove NA cell

x_high_train, x_high_test, y_high_train, y_high_test = train_test_split(x_high, y_high, test_size = 0.1)

# low data
data_low["Predictin"] = data_low[["Low_price"]].shift(-prediction_step)
x_low = np.array(data_low.drop(["Predictin"], 1))
x_low = x_low[:len(data_low)-prediction_step] # remove NA cell
y_low = np.array(data_low["Predictin"])
y_low = y_low[:-prediction_step] # remove NA cell

x_low_train, x_low_test, y_low_train, y_low_test = train_test_split(x_low, y_low, test_size = 0.1)

# close data
data_close["Predictin"] = data_close[["Close_price"]].shift(-prediction_step)
x_close = np.array(data_close.drop(["Predictin"], 1))
x_close = x_close[:len(data_close)-prediction_step] # remove NA cell
y_close = np.array(data_close["Predictin"])
y_close = y_close[:-prediction_step] # remove NA cell

x_close_train, x_close_test, y_close_train, y_close_test = train_test_split(x_close, y_close, test_size = 0.1)

# create an array for predicted value
predicted_step_array_high = np.array(data_high.drop(["Predictin"], 1))[-prediction_step:]
predicted_step_array_low = np.array(data_low.drop(["Predictin"], 1))[-prediction_step:]
predicted_step_array_close = np.array(data_close.drop(["Predictin"], 1))[-prediction_step:]
print("Successful")

TypeError: drop() takes from 1 to 2 positional arguments but 3 were given

In [5]:
# predict high price

# create a svr model
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}
svr_model = SVR(kernel = "rbf", C = 1e4, gamma = 0.00001)
svr_model.fit(x_high_train, y_high_train)

# model accuracy
model_confidence_high = svr_model.score(x_high_test, y_high_test)
model_prediction_high = svr_model.predict(x_high_test)

model_prediction_high = (model_prediction_high * s_high) + m_high
y_high_test = (y_high_test * s_high) + m_high
rmse_high = np.sqrt(np.mean(model_prediction_high - y_high_test) ** 2)
R2_high = model_confidence_high

# plt.plot(y_test)
# plt.plot(model_prediction)
# plt.show()

next_steps_high = svr_model.predict(predicted_step_array_high)
next_steps_high = (next_steps_high * s_high) + m_high
print("Successful")

Successful


In [6]:
# predict low price

# create a svr model
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}
svr_model = SVR(kernel = "rbf", C = 1e4, gamma = 0.00001)
svr_model.fit(x_low_train, y_low_train)

# model accuracy
model_confidence_low = svr_model.score(x_low_test, y_low_test)
model_prediction_low = svr_model.predict(x_low_test)

model_prediction_low = (model_prediction_low * s_low) + m_low
y_low_test = (y_low_test * s_low) + m_low
rmse_low = np.sqrt(np.mean(model_prediction_low - y_low_test) ** 2)
R2_low = model_confidence_low

# plt.plot(y_test)
# plt.plot(model_prediction)
# plt.show()

next_steps_low = svr_model.predict(predicted_step_array_low)
next_steps_low = (next_steps_low * s_low) + m_low
print("Successful")

Successful


In [7]:
# predict close price

# create a svr model
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}
svr_model = SVR(kernel = "rbf", C = 1e4, gamma = 0.00001)
svr_model.fit(x_close_train, y_close_train)

# model accuracy
model_confidence_close = svr_model.score(x_close_test, y_close_test)
model_prediction_close = svr_model.predict(x_close_test)

model_prediction_close = (model_prediction_close * s_close) + m_close
y_close_test = (y_close_test * s_close) + m_close
rmse_close = np.sqrt(np.mean(model_prediction_close - y_close_test) ** 2)
R2_close = model_confidence_close

# plt.plot(y_test)
# plt.plot(model_prediction)
# plt.show()

next_steps_close = svr_model.predict(predicted_step_array_close)
next_steps_close = (next_steps_close * s_close) + m_close
print("Successful")

Successful


In [8]:
# print results
print("R2 High : ", R2_high)
print("RMSE High : ", rmse_high)
print("R2 Low : ", R2_low)
print("RMSE Low : ", rmse_low)
print("R2 Close : ", R2_close)
print("RMSE Close : ", rmse_close)
print("*****************************************")
print("High price :")
print("Previous high price : ", a_high)
print("Next step high price : ", next_steps_high)
print("Chaing high price in % : ", ((a_high * 100)/next_steps_high)-100)
print("*****************************************")
print("Low price :")
print("Previous low price : ", a_low)
print("Next step low price : ", next_steps_low)
print("Chaing low price in % : ", ((a_low * 100)/next_steps_low)-100)
print("*****************************************")
print("Close price :")
print("Previous close price : ", a_close)
print("Next step close price : ", next_steps_close)
print("Chaing close price in % : ", ((a_close * 100)/next_steps_close)-100)



del(data_high, data_low, data_close, m_high, m_low, m_close, model_prediction_high
    , model_prediction_low, model_prediction_close, predicted_step_array_high, 
    predicted_step_array_low, predicted_step_array_close, prediction_step, x_high, 
    x_low, x_close, x_high_test, x_low_test, x_close_test, x_high_train, x_low_train, 
    x_close_train, y_high, y_low, y_close, y_high_test, y_low_test, y_close_test, 
    y_high_train, y_low_train, y_close_train, s_high, s_low, s_close, svr_model)

R2 High :  0.9967649118078642
RMSE High :  0.03721276041980716
R2 Low :  0.9958711949857061
RMSE Low :  0.036028603174164585
R2 Close :  0.9966507569121401
RMSE Close :  0.040597093302772086
*****************************************
High price :
Previous high price :  3.75054
Next step high price :  [3.74694842]
Chaing high price in % :  [0.09585356]
*****************************************
Low price :
Previous low price :  3.65417
Next step low price :  [3.63795004]
Chaing low price in % :  [0.44585452]
*****************************************
Close price :
Previous close price :  3.74604
Next step close price :  [3.73460939]
Chaing close price in % :  [0.30607242]
